In [1]:
import pandas as pd
import numpy as np
import ast

DATA_FOLDER = "../../DATA/"

In [2]:
FINAL_TEST = pd.read_parquet(DATA_FOLDER + '/RAW/test.parq')

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
FINAL_TEST['attack_time'] = pd.to_datetime(FINAL_TEST['attack_time']).dt.day
FINAL_TEST[['attack_time']] = scaler.fit_transform(FINAL_TEST[['attack_time']])
FINAL_TEST[['watcher_as_num']] = scaler.fit_transform(FINAL_TEST[['watcher_as_num']])
FINAL_TEST[['attacker_as_num']] = scaler.fit_transform(FINAL_TEST[['attacker_as_num']])
FINAL_TEST[['watcher_uuid_enum']] = scaler.fit_transform(FINAL_TEST[['watcher_uuid_enum']])

In [3]:
shodan = pd.read_parquet(DATA_FOLDER + "/PROCESSING/NEW_SHODAN_FEATURES.parq")
shodan

,attacker_ip_enum,port_count,HAS_TCP?,HAS_UDP?,risky_ports
0,5915,0,False,False,False
1,3325,1,False,False,True
2,8416,0,False,False,False
3,1213,0,False,False,False
4,9185,0,False,False,False
...,...,...,...,...,...
197674,196812,2,False,False,False
197675,191141,0,False,False,False
197676,195077,3,False,False,True
197677,198002,0,False,False,False


In [4]:
FINAL_TEST = pd.merge(FINAL_TEST, shodan, left_on='attacker_ip_enum', right_on='attacker_ip_enum', how='left')
del shodan

In [5]:
FINAL_TEST[["protocol", "attack_type"]] = FINAL_TEST['attack_type'].str.split(':', expand=True)
FINAL_TEST

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,port_count,HAS_TCP?,HAS_UDP?,risky_ports,protocol
0,1.783153,US,-0.123950,IMH-IAD,IN,-0.488094,DIGITALOCEAN-ASN,spam,-0.985816,7696,17,False,False,True,http
1,1.783153,US,-0.123950,IMH-IAD,IN,-0.488094,DIGITALOCEAN-ASN,exploit,-0.985816,7696,17,False,False,True,http
2,1.783153,US,-0.123950,IMH-IAD,IN,-0.488094,DIGITALOCEAN-ASN,bruteforce,-0.985816,7696,17,False,False,True,http
3,1.783153,US,-0.123950,IMH-IAD,IN,-0.488094,DIGITALOCEAN-ASN,bruteforce,-0.985816,7696,17,False,False,True,http
4,1.783153,US,3.033962,GOOGLE-CLOUD-PLATFORM,US,-0.098828,PONYNET,bruteforce,-0.985679,7543,1,False,False,False,http
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18682292,0.959273,DE,-0.509016,ENTEGA Medianet GmbH,DE,-0.593662,Deutsche Telekom AG,scan,0.721205,198081,0,False,False,False,http
18682293,0.959273,DE,-0.509016,ENTEGA Medianet GmbH,DE,-0.593662,Deutsche Telekom AG,scan,0.721205,198081,0,False,False,False,http
18682294,0.959273,DE,-0.509016,ENTEGA Medianet GmbH,DE,-0.593662,Deutsche Telekom AG,scan,0.721205,198081,0,False,False,False,http
18682295,0.959273,DE,-0.509016,ENTEGA Medianet GmbH,DE,-0.593662,Deutsche Telekom AG,scan,0.721205,198081,0,False,False,False,http


In [6]:
FINAL_TEST["joined_countries"] = FINAL_TEST["attacker_country"].astype(str) + ":" + FINAL_TEST["watcher_country"].astype(str)
FINAL_TEST

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,port_count,HAS_TCP?,HAS_UDP?,risky_ports,protocol,joined_countries
0,1.783153,US,-0.123950,IMH-IAD,IN,-0.488094,DIGITALOCEAN-ASN,spam,-0.985816,7696,17,False,False,True,http,IN:US
1,1.783153,US,-0.123950,IMH-IAD,IN,-0.488094,DIGITALOCEAN-ASN,exploit,-0.985816,7696,17,False,False,True,http,IN:US
2,1.783153,US,-0.123950,IMH-IAD,IN,-0.488094,DIGITALOCEAN-ASN,bruteforce,-0.985816,7696,17,False,False,True,http,IN:US
3,1.783153,US,-0.123950,IMH-IAD,IN,-0.488094,DIGITALOCEAN-ASN,bruteforce,-0.985816,7696,17,False,False,True,http,IN:US
4,1.783153,US,3.033962,GOOGLE-CLOUD-PLATFORM,US,-0.098828,PONYNET,bruteforce,-0.985679,7543,1,False,False,False,http,US:US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18682292,0.959273,DE,-0.509016,ENTEGA Medianet GmbH,DE,-0.593662,Deutsche Telekom AG,scan,0.721205,198081,0,False,False,False,http,DE:DE
18682293,0.959273,DE,-0.509016,ENTEGA Medianet GmbH,DE,-0.593662,Deutsche Telekom AG,scan,0.721205,198081,0,False,False,False,http,DE:DE
18682294,0.959273,DE,-0.509016,ENTEGA Medianet GmbH,DE,-0.593662,Deutsche Telekom AG,scan,0.721205,198081,0,False,False,False,http,DE:DE
18682295,0.959273,DE,-0.509016,ENTEGA Medianet GmbH,DE,-0.593662,Deutsche Telekom AG,scan,0.721205,198081,0,False,False,False,http,DE:DE


In [7]:
FINAL_TEST['protocol'] = FINAL_TEST['protocol'].astype('category')
FINAL_TEST['attack_type'] = FINAL_TEST['attack_type'].astype('category')
FINAL_TEST['joined_countries'] = FINAL_TEST['joined_countries'].astype('category')
FINAL_TEST["attacker_ip_enum"] = FINAL_TEST["attacker_ip_enum"].astype(np.int64)
FINAL_TEST.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18682297 entries, 0 to 18682296
Data columns (total 16 columns):
 #   Column             Dtype   
---  ------             -----   
 0   attack_time        float64 
 1   watcher_country    category
 2   watcher_as_num     float32 
 3   watcher_as_name    category
 4   attacker_country   category
 5   attacker_as_num    float32 
 6   attacker_as_name   category
 7   attack_type        category
 8   watcher_uuid_enum  float64 
 9   attacker_ip_enum   int64   
 10  port_count         int64   
 11  HAS_TCP?           bool    
 12  HAS_UDP?           bool    
 13  risky_ports        bool    
 14  protocol           category
 15  joined_countries   category
dtypes: bool(3), category(7), float32(2), float64(2), int64(2)
memory usage: 980.5 MB


In [8]:
def fill_nan_with_mean(df):
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            mean_value = df[column].mean()
            df[column].fillna(mean_value, inplace=True)
        elif pd.api.types.is_categorical_dtype(df[column]):
            mode_value = df[column].mode().iloc[0]  # Mode for categorical columns
            df[column].fillna(mode_value, inplace=True)

fill_nan_with_mean(FINAL_TEST)

C:\Users\Francisco\AppData\Local\Temp\ipykernel_9268\1584703964.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mean_value, inplace=True)
C:\Users\Francisco\AppData\Local\Temp\ipykernel_9268\1584703964.py:6: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(df[column]):
C:\Users\Francisco\AppData\Local\Temp\ipykernel_9268\1584703964.py:8: FutureWarning: A value is tryin

In [9]:
FINAL_TEST.to_parquet(DATA_FOLDER + '/PROCESSING/PREPEARED_COMP_TEST.parq', index=True)

In [10]:
FINAL_TEST

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,port_count,HAS_TCP?,HAS_UDP?,risky_ports,protocol,joined_countries
0,1.783153,US,-0.123950,IMH-IAD,IN,-0.488094,DIGITALOCEAN-ASN,spam,-0.985816,7696,17,False,False,True,http,IN:US
1,1.783153,US,-0.123950,IMH-IAD,IN,-0.488094,DIGITALOCEAN-ASN,exploit,-0.985816,7696,17,False,False,True,http,IN:US
2,1.783153,US,-0.123950,IMH-IAD,IN,-0.488094,DIGITALOCEAN-ASN,bruteforce,-0.985816,7696,17,False,False,True,http,IN:US
3,1.783153,US,-0.123950,IMH-IAD,IN,-0.488094,DIGITALOCEAN-ASN,bruteforce,-0.985816,7696,17,False,False,True,http,IN:US
4,1.783153,US,3.033962,GOOGLE-CLOUD-PLATFORM,US,-0.098828,PONYNET,bruteforce,-0.985679,7543,1,False,False,False,http,US:US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18682292,0.959273,DE,-0.509016,ENTEGA Medianet GmbH,DE,-0.593662,Deutsche Telekom AG,scan,0.721205,198081,0,False,False,False,http,DE:DE
18682293,0.959273,DE,-0.509016,ENTEGA Medianet GmbH,DE,-0.593662,Deutsche Telekom AG,scan,0.721205,198081,0,False,False,False,http,DE:DE
18682294,0.959273,DE,-0.509016,ENTEGA Medianet GmbH,DE,-0.593662,Deutsche Telekom AG,scan,0.721205,198081,0,False,False,False,http,DE:DE
18682295,0.959273,DE,-0.509016,ENTEGA Medianet GmbH,DE,-0.593662,Deutsche Telekom AG,scan,0.721205,198081,0,False,False,False,http,DE:DE
